In [10]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pickle
import pyprind

In [2]:
PATH_DATA = "V:/Programmazione/Amazon/"

In [5]:
df = pd.read_pickle("%sAmazonDataProject.pkl" % PATH_DATA)

In [11]:
def deleteStopwords(listReviews):
    stopWords = set(stopwords.words('english'))
    res = ""
    for s in pyprind.prog_bar(listReviews):
        text = nltk.word_tokenize(s)
        for t in text:
            if t not in stopWords:
                res+="%s " % t
    return res

In [12]:
all_reviews_without_stopwords = deleteStopwords(df.review_body.tolist())

KeyboardInterrupt: 

In [ ]:
categories = df.product_category.unique().tolist()
print(deleteStopwords(df[df.product_category==categories[0]].review_body.tolist()))

0% [#                             ] 100% | ETA: 00:26:17